In [1]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

# DATABASE_LOCATION = 'my_played_tracks.sqlite'
USER_ID = '227dt3g18rrzyco2j7p2boy2ly'
TOKEN = 'BQC2F8FNt36AMHHKG5Xt-K91SJmYS6If3seYHNlkLzGVTirCwDw4U5Nk9RRXEhJ0f8H8oaETWQF6MQJGkMO9yZxva9IxjP5lZ0X7pzpcFTuDIcGtBPoirO-8Z5pcXlX1yp_XpxeaomkxGFHbf1Hy-xdwfeF7yHfzNSPm-vs5'

In [2]:
headers = {
    "Accept":"application/json",
    "Content-Type":"application/json",
    "Authorization":"Bearer {token}".format(token=TOKEN)
}
    

In [3]:
r = requests.get("https://api.spotify.com/v1/me/player/recently-played",headers = headers)
response = r.json()

In [44]:
my_song_list = []

for song in response['items']:
    artist_id = song['track']['artists'][0]['id']
    artist_name = song['track']['artists'][0]['name']
    artist_link = song['track']['artists'][0]['external_urls']['spotify'] 
    album_id = song['track']['album']['id']
    album_name = song['track']['album']['name']
    album_link = song['track']['album']['external_urls']['spotify']
    song_id = song['track']['id']
    song_name = song['track']['name']
    song_link = song['track']['external_urls']['spotify']
    duration_ms = song['track']['duration_ms']
    popularity = song['track']['popularity']
    disc_number = song['track']['disc_number']
    played_at = song['played_at']
    
    song_dic = {'artist_id': artist_id,
                     'artist_name':artist_name,
                     'artist_link':artist_link,
                     'album_id':album_id,
                     'album_name':album_name,
                     'album_link':album_link,
                     'song_id':song_id,
                     'song_name':song_name,
                     'song_link':song_link,
                     'duration_ms':duration_ms,
                     'popularity':popularity,
                     'disc_number':disc_number,
                     'played_at':played_at
                     }
    
    my_song_list.append(song_dic)
    # print(my_song_list)
    df = pd.DataFrame(my_song_list)

In [45]:
df

,album_id,album_link,album_name,artist_id,artist_link,artist_name,disc_number,duration_ms,played_at,popularity,song_id,song_link,song_name
0,5Eevxp2BCbWq25ZdiXRwYd,https://open.spotify.com/album/5Eevxp2BCbWq25Z...,One More Light,6XyY86QOPPrYVGvF9ch6wz,https://open.spotify.com/artist/6XyY86QOPPrYVG...,Linkin Park,1,255066,2021-12-22T01:12:06.780Z,74,3xXBsjrbG1xQIm1xv1cKOt,https://open.spotify.com/track/3xXBsjrbG1xQIm1...,One More Light
1,54XqIHrxcpeDw83U1Wv39c,https://open.spotify.com/album/54XqIHrxcpeDw83...,Constellation Bay,6azdwGecDNGbIpCpOOdWsG,https://open.spotify.com/artist/6azdwGecDNGbIp...,admo,1,232900,2021-12-22T01:07:55.049Z,20,0Vi9jO0QjML3HBtw9kGVoI,https://open.spotify.com/track/0Vi9jO0QjML3HBt...,Jetlag
2,0G3VsuxqbopJjcjoHIbsbg,https://open.spotify.com/album/0G3VsuxqbopJjcj...,Sun In Your Eyes (Spencer Brown Remix),10gzBoINW3cLJfZUka8Zoe,https://open.spotify.com/artist/10gzBoINW3cLJf...,Above & Beyond,1,355200,2021-12-22T00:57:55.442Z,52,7cORfHvrFDJyMF0LaA5VRt,https://open.spotify.com/track/7cORfHvrFDJyMF0...,Sun In Your Eyes - Spencer Brown Edit
3,5cb5ZgNp1dLbymi2lXWrVm,https://open.spotify.com/album/5cb5ZgNp1dLbymi...,Orbital,4c52ieTjtWXVaW4RCtXCHY,https://open.spotify.com/artist/4c52ieTjtWXVaW...,Voyager,1,261631,2021-12-22T00:51:47.127Z,41,4A4DryuyUKLQOhuiFKBaPF,https://open.spotify.com/track/4A4DryuyUKLQOhu...,Intelestellar
4,0GQryILWlNCRo1ifSa55qs,https://open.spotify.com/album/0GQryILWlNCRo1i...,Rêve Lucide,4giO85Ma0hNO0wBWLFtJsS,https://open.spotify.com/artist/4giO85Ma0hNO0w...,LÜNE,1,189600,2021-12-22T00:46:13.077Z,37,0cBnnLmUdJBDpPYqk92RdJ,https://open.spotify.com/track/0cBnnLmUdJBDpPY...,Soleil Bleu
5,36Mgk6gWyWrjbGuJXFxRTV,https://open.spotify.com/album/36Mgk6gWyWrjbGu...,Dive,3DMRmCWZFPz3IX8LUgoLVb,https://open.spotify.com/artist/3DMRmCWZFPz3IX...,Krosia,1,237362,2021-12-22T00:43:07.234Z,38,2FCt5PvJPKL41hxSHMyIrO,https://open.spotify.com/track/2FCt5PvJPKL41hx...,Dive
6,1RHa1VdX6lsLbeedgsV1cb,https://open.spotify.com/album/1RHa1VdX6lsLbee...,Pop Food,1vVHevk2PD45epYnDi9CCc,https://open.spotify.com/artist/1vVHevk2PD45ep...,Jack Stauber,1,185223,2021-12-22T00:38:25.798Z,77,6SZwj6cjc8JQbkrr8cbIUn,https://open.spotify.com/track/6SZwj6cjc8JQbkr...,Oh Klahoma
7,3PX5RP9vQ1q2aDRPQTNUBn,https://open.spotify.com/album/3PX5RP9vQ1q2aDR...,Anemoia,1Xq5wasmlEwC6TqCqJtP5o,https://open.spotify.com/artist/1Xq5wasmlEwC6T...,Attom,1,283090,2021-12-22T00:35:24.546Z,29,4JIOES7JGwGiFuvwDnHch6,https://open.spotify.com/track/4JIOES7JGwGiFuv...,Anemoia
8,5S3gls8Kjn8KVmqlIDEBbO,https://open.spotify.com/album/5S3gls8Kjn8KVmq...,The Number of the Beast (2015 Remaster),6mdiAmATAx73kdxrNrnlao,https://open.spotify.com/artist/6mdiAmATAx73kd...,Iron Maiden,1,396760,2021-12-22T00:30:41.881Z,54,51S9pXdACCrhputBUl05Cs,https://open.spotify.com/track/51S9pXdACCrhput...,22 Acacia Avenue - 2015 Remaster
9,1fyLNx6wxgDA59wFInnyup,https://open.spotify.com/album/1fyLNx6wxgDA59w...,Thank You,2UazAtjfzqBF0Nho2awK4z,https://open.spotify.com/artist/2UazAtjfzqBF0N...,Stone Temple Pilots,1,230293,2021-12-22T00:16:57.705Z,68,3ftHrCjsTUPLgI48m67byk,https://open.spotify.com/track/3ftHrCjsTUPLgI4...,Plush - Acoustic
